In [1]:
import cv2
import pandas as pd
import numpy as np
import csv
from datetime import datetime, timedelta
import os
from spmfunctions.misc_tools import detector_status, phase_status, overlap_status, comb_gyr_det


In [50]:
import tkinter as tk
from tkinter import simpledialog, messagebox

class VideoProcessor:
    def __init__(self):
        self.cap = None
        self.fps = 0
        self.width = 0
        self.height = 0
        self.shapes = []
        self.current_shape = []
        self.mode = 'loop'
        self.color = (0, 255, 0)
        self.input_val = 1
        self.phase = 1

    def read_video(self, video_path):
        """Read video and get properties"""
        self.cap = cv2.VideoCapture(video_path)
        self.fps = self.cap.get(cv2.CAP_PROP_FPS)
        self.width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        return self.cap, self.fps, self.width, self.height

    def draw_shape(self, img, shape):
        """Draw a shape on image"""
        if shape['type'] == 'loop':
            pts = np.array(shape['points'], dtype=np.int32)
            cv2.polylines(img, [pts], isClosed=True, color=shape['color'], thickness=2)
        elif shape['type'] == 'stopbar':
            pt1, pt2 = shape['points']
            cv2.line(img, pt1, pt2, color=(0, 0, 255), thickness=2)

    def mouse_callback(self, event, x, y, flags, param):
        """Handle mouse events for drawing"""
        if event == cv2.EVENT_LBUTTONDOWN:
            self.current_shape.append((x, y))
            if len(self.current_shape) == 4 and self.mode == 'loop':
                # Compute 4th corner for rotated rectangle
                self.shapes.append({
                    'type': 'loop',
                    'points': list(self.current_shape),
                    'color': self.color,
                    'input': self.input_val
                })
                self.current_shape = []
            elif len(self.current_shape) == 2 and self.mode == 'stopbar':
                self.shapes.append({
                    'type': 'stopbar',
                    'points': list(self.current_shape),
                    'phase': self.phase
                })
                self.current_shape = []

    def draw_shapes_interface(self):
        """Interactive interface for drawing shapes with Tkinter input dialogs"""

        if not self.cap:
            raise ValueError("Video not loaded")
        
        ret, first_frame = self.cap.read()
        if not ret:
            raise ValueError("Cannot read first frame")
        
        # Reset to beginning
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        
        image = first_frame.copy()
        cv2.namedWindow("Draw Shapes")
        cv2.setMouseCallback("Draw Shapes", self.mouse_callback)

        # Tkinter root for dialogs
        root = tk.Tk()
        root.withdraw()

        # Create a small always-on-top Tkinter window for instructions
        instruction_window = tk.Toplevel()
        instruction_window.title("Draw Shapes Instructions")
        instruction_window.attributes('-topmost', True)
        instruction_window.resizable(False, False)
        instruction_text = (
            "Instructions:\n"
            "- Press 'l' to switch to loop mode (4 points)\n"
            "- Press 's' to switch to stop bar mode (2 points)\n"
            "- Press 'c' to change color (for loops)\n"
            "- Press 'i' to set input value (for loops)\n"
            "- Press 'p' to set phase value (for stop bars)\n"
            "- Press 'u' to undo last action\n"
            "- Click to draw shapes\n"
            "- Press 'q' when finished"
        )
        label = tk.Label(instruction_window, text=instruction_text, justify='left', padx=10, pady=10)
        label.pack()
        # Position the window at the top right corner
        instruction_window.update_idletasks()
        screen_width = instruction_window.winfo_screenwidth()
        window_width = instruction_window.winfo_width()
        instruction_window.geometry(f"+{screen_width - window_width - 40}+40")

        colors = {
            'Green': (0, 255, 0),
            'Blue': (255, 0, 0),
            'Red': (0, 0, 255),
            'Yellow': (255, 255, 0),
            'Magenta': (255, 0, 255),
            'Cyan': (0, 255, 255),
            'Black': (0, 0, 0)
        }
        color_index = 0

        while True:
            img_copy = image.copy()
            
            # Draw existing shapes
            for shape in self.shapes:
                self.draw_shape(img_copy, shape)
            
            # Draw current shape being created
            if len(self.current_shape) > 0:
                for pt in self.current_shape:
                    cv2.circle(img_copy, pt, 5, (0, 0, 0), -1)
                if len(self.current_shape) == 2:
                    cv2.line(img_copy, self.current_shape[0], self.current_shape[1], (0, 0, 0), 2)
                elif len(self.current_shape) == 3:
                    cv2.line(img_copy, self.current_shape[1], self.current_shape[2], (0, 0, 0), 2)
                elif len(self.current_shape) == 4:
                    pts = np.array(self.current_shape, dtype=np.int32)
                    cv2.polylines(img_copy, [pts], isClosed=True, color=(0, 0, 0), thickness=1)

            # Display current mode and values
            mode_text = f"Mode: {self.mode}"
            if self.mode == 'loop':
                color_name = next((name for name, rgb in colors.items() if rgb == self.color), str(self.color))
                mode_text += f" | Color: {color_name} | Input: {self.input_val}"
            else:
                mode_text += f" | Phase: {self.phase}"
            cv2.putText(img_copy, mode_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

            cv2.imshow("Draw Shapes", img_copy)
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('q'):
                break
            elif key == ord('l'):
                self.mode = 'loop'
            elif key == ord('s'):
                self.mode = 'stopbar'
            elif key == ord('c'):
                color_names = list(colors.keys())
                color_index = (color_index + 1) % len(color_names)
                color_name = color_names[color_index]
                self.color = colors[color_name]
            elif key == ord('i'):
                inp = simpledialog.askinteger("Input Value", "Enter input value (1-64):", minvalue=1, maxvalue=64)
                if inp is not None:
                    self.input_val = inp
            elif key == ord('p'):
                phase_input = simpledialog.askstring("Phase Value", "Enter phase value (1-16 or A-P):")
                if phase_input:
                    phase_input = phase_input.strip().upper()
                    if phase_input.isdigit():
                        phase = int(phase_input)
                        if 1 <= phase <= 16:
                            self.phase = phase
                        else:
                            messagebox.showerror("Phase", "Phase must be between 1-16")
                    elif len(phase_input) == 1 and 'A' <= phase_input <= 'P':
                        self.phase = f"OL{phase_input}"
                    else:
                        messagebox.showerror("Phase", "Phase must be between 1-16 or A-P")
            elif key == ord('u'):  # Undo last action
                if self.current_shape:
                    self.current_shape.pop()
                    print("Undone: Removed last point.")
                elif self.shapes:
                    removed_shape = self.shapes.pop()
                    shape_type = removed_shape['type'].title()
                    print(f"Undone: Removed last {shape_type} shape.")
                else:
                    print("Nothing to undo.")

        cv2.destroyAllWindows()
        root.destroy()
        return self.shapes

    def save_shapes_to_csv(self, filename):
        """Save shapes to CSV file"""
        with open(filename, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['type', 'points', 'color', 'input', 'phase', 'video_width', 'video_height'])
            for shape in self.shapes:
                points_str = ';'.join([f"{pt[0]},{pt[1]}" for pt in shape['points']])
                color_str = f"{shape.get('color', (0,0,0))[0]},{shape.get('color', (0,0,0))[1]},{shape.get('color', (0,0,0))[2]}"
                writer.writerow([
                    shape['type'],
                    points_str,
                    color_str,
                    shape.get('input', ''),
                    shape.get('phase', ''),
                    self.width,
                    self.height
                ])

    def load_shapes_from_csv(self, filename):
        """Load shapes from CSV file"""
        shapes = []
        if os.path.exists(filename):
            with open(filename, 'r') as f:
                reader = csv.DictReader(f)
                for row in reader:
                    points = []
                    for pt_str in row['points'].split(';'):
                        x, y = map(int, pt_str.split(','))
                        points.append((x, y))
                    
                    color = tuple(map(int, row['color'].split(','))) if row['color'] else (0, 255, 0)
                    
                    shape = {
                        'type': row['type'],
                        'points': points,
                        'color': color,
                        'input': int(row['input']) if row['input'] else None,
                        'phase': row['phase'] if row['phase'] else None
                    }
                    shapes.append(shape)
        self.shapes = shapes
        return shapes

    def load_and_process_data(self, pickle_path, start_time_str, end_time_str, time_step=0.1):
        """Load and process data from pickle file with datetime filtering"""
        df = pd.read_pickle(pickle_path)
        
        # Convert start and end times to datetime
        start_dt = pd.to_datetime(start_time_str)
        end_dt = pd.to_datetime(end_time_str)
        df_start = start_dt - pd.Timedelta(hours=1)  # 1 hour before start_dt
        df_end = end_dt + pd.Timedelta(hours=1)      # 1 hour after end_dt
        
        # Filter data to relevant time range
        df = df[(df['TS_start'] >= df_start) & (df['TS_start'] <= df_end)]
        
        if df.empty:
            print("Warning: No data found in the specified time range")
            return df
        
        # Get relevant phases and detectors from shapes
        relevant_phases = list(set([s['phase'] for s in self.shapes if s['type'] == 'stopbar' and s['phase'] is not None and "OL" not in str(s['phase'])]))
        
        # Map overlap names like "OLA", "OLB", etc. to numbers: "OLA"->1, "OLB"->2, ...
        overlap_map = {f"OL{chr(ord('A') + i)}": i + 1 for i in range(26)}
        relevant_overlaps = []
        for s in self.shapes:
            if s['type'] == 'stopbar' and s['phase'] is not None and "OL" in str(s['phase']):
                phase_str = str(s['phase'])
                mapped_val = overlap_map.get(phase_str)
                if mapped_val is not None:
                    relevant_overlaps.append(mapped_val)
        relevant_overlaps = list(set(relevant_overlaps))
        
        relevant_detectors = list(set([s['input'] for s in self.shapes if s['type'] == 'loop' and s['input'] is not None]))


        df = comb_gyr_det(df)
        df = detector_status(df, relevant_detectors)
        df = phase_status(df, relevant_phases)
        df = overlap_status(df, relevant_overlaps)
        
        # Select relevant columns
        cols_to_keep = ['TS_start']
        for ph in relevant_phases:
            col_name = f'Ph {ph} Status'
            if col_name in df.columns:
                cols_to_keep.append(col_name)
        for ol in relevant_overlaps:
            col_name = f'OL{chr(ord("A") + ol - 1)} Status' if ol <= 26 else f'OL{ol} Status'
            if col_name in df.columns:
                cols_to_keep.append(col_name)
        for det in relevant_detectors:
            col_name = f'Det {det} Status'
            if col_name in df.columns:
                cols_to_keep.append(col_name)

        
        df = df[cols_to_keep]
        df.drop_duplicates(subset=['TS_start'], inplace=True, keep='last')
        df.sort_values('TS_start', inplace=True)

        # Expand timestamps
        if not df.empty:
            expanded_times = pd.date_range(start=start_dt, end=end_dt, freq=f'{int(time_step*1000)}ms')
            expanded_df = pd.DataFrame({'TS_start': expanded_times})

            # Merge and forward-fill
            merged_df = pd.merge_asof(expanded_df, df, on='TS_start', direction='forward')
            return merged_df
        
        df.to_csv('expanded_data.csv', index=False)
        return df

    def extract_frames_at_intervals(self, start_dt, end_dt, interval=0.1):
        """Extract frames at specified datetime intervals"""
        frames = []
        timestamps = []

        # Convert datetime to seconds from video start
        video_start_time = 0.0  # Assuming video starts at 0 seconds
        
        current_dt = start_dt
        while current_dt <= end_dt:
            # Calculate time in seconds from video start
            time_diff = (current_dt - start_dt).total_seconds()
            current_time = video_start_time + time_diff
            
            frame_idx = int(current_time * self.fps)
            self.cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
            ret, frame = self.cap.read()
            if ret:
                frames.append(frame.copy())
                timestamps.append(current_dt)
            current_dt += pd.Timedelta(seconds=interval)

        return frames, timestamps

    def overlay_shapes(self, frame, row_data):
        """Overlay shapes on frame based on data"""
        for shape in self.shapes:
            if shape['type'] == 'loop' and shape['input'] is not None:
                det_col = f"Det {shape['input']} Status"
                status = row_data.get(det_col, 'na') if det_col in row_data else 'na'

                if pd.isna(status) or status == 'na':
                    outline_color = (0,0,0)
                    fill_color = None
                    alpha = 0
                elif status == 'Off':
                    outline_color = shape['color']
                    fill_color = None
                    alpha = 0
                elif status == 'On':
                    outline_color = (255, 255, 255)
                    fill_color = shape['color']
                    alpha = 0.2

                pts = np.array(shape['points'], dtype=np.int32)
                if alpha > 0:
                    overlay = frame.copy()
                    cv2.fillPoly(overlay, [pts], fill_color)
                    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
                cv2.polylines(frame, [pts], isClosed=True, color=outline_color, thickness=1)

            elif shape['type'] == 'stopbar' and shape['phase'] is not None:
                # Determine if phase is overlap (e.g., 'OLA', 'OLB', ...) or integer
                phase_val = shape['phase']
                # Check if phase is integer-like (int or string of digits)
                if isinstance(phase_val, int) or (isinstance(phase_val, str) and phase_val.isdigit()):
                    ph_col = f"Ph {phase_val} Status"
                else:
                    ph_col = f"{phase_val} Status"
                status = row_data.get(ph_col, 'na') if ph_col in row_data else 'na'
                color_map = {'R': (0, 0, 255), 'Y': (0, 255, 255), 'G': (0, 255, 0), 'Rc': (0, 0, 255), 'na': (128, 128, 128)}
                color = color_map.get(status, (0, 0, 0))  # Default to black if status not recognized
                pt1, pt2 = shape['points']
                cv2.line(frame, pt1, pt2, color, thickness=3)

    def write_video(self, frames, output_path, fps):
        """Write frames to output video"""
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (self.width, self.height))
        for frame in frames:
            out.write(frame)
        out.release()

    def process_video(self, video_path, pickle_path, output_path, shape_csv, time_step=0.1, 
                     start_time_str=None, end_time_str=None):
        """Main processing function with datetime support"""
        # Read video
        self.read_video(video_path)
        
        # Load or create shapes
        if os.path.exists(shape_csv):
            self.load_shapes_from_csv(shape_csv)
            print(f"Loaded {len(self.shapes)} shapes from {shape_csv}")
        else:
            print("No shape file found. Starting drawing interface...")
            self.draw_shapes_interface()
            self.save_shapes_to_csv(shape_csv)
            print(f"Saved {len(self.shapes)} shapes to {shape_csv}")
        
        # Validate datetime inputs
        if not start_time_str or not end_time_str:
            raise ValueError("Start and end times are required")
        
        try:
            start_dt = pd.to_datetime(start_time_str)
            end_dt = pd.to_datetime(end_time_str)
        except Exception as e:
            raise ValueError(f"Invalid datetime format: {e}")
        
        # Load and process data
        df = self.load_and_process_data(pickle_path, start_time_str, end_time_str, time_step)
        if df.empty:
            print("No data to process in the specified time range")
            return

        # Extract frames
        frames, timestamps = self.extract_frames_at_intervals(start_dt, end_dt, time_step)
        print(f"Extracted {len(frames)} frames")

        # Process frames with overlays
        for i, (frame, ts) in enumerate(zip(frames, timestamps)):
            if not df.empty:
                # Find closest data row
                closest_idx = (df['TS_start'] - ts).abs().argmin()
                row_data = df.iloc[closest_idx]
                self.overlay_shapes(frame, row_data)
            
            # Add timestamp to frame
            timestamp_str = ts.strftime("%Y-%m-%d %H:%M:%S.%f")[:-5]  # Show up to tenth of second
            cv2.putText(frame, timestamp_str, (10, self.height - 10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # Write output video
        output_fps = 1.0 / time_step
        self.write_video(frames, output_path, output_fps)
        print(f"Output video saved to {output_path}")


In [30]:
def main(video_path, pickle_path, output_path, shape_csv, start_time_str, end_time_str, time_step=0.1):
    """Main function"""
    processor = VideoProcessor()
   
    if not start_time_str or not end_time_str:
        print("Start and end datetime are required!")
        return

    # Process video
    try:
        processor.process_video(video_path, pickle_path, output_path, shape_csv, 
                              time_step, start_time_str, end_time_str)
        print("Processing completed successfully!")
    except Exception as e:
        print(f"Error during processing: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
#video_path = input("Enter video path: ").strip() or "input.mp4"
#pickle_path = input("Enter pickle data path: ").strip() or "data.pkl"
#output_path = input("Enter output video path: ").strip() or "output.mp4"
#shape_csv = input("Enter shape CSV path: ").strip() or "shapes.csv"
#start_time_str = input("Enter start datetime (yyyy-mm-dd hh:mm:ss.0): ").strip()
#end_time_str = input("Enter end datetime (yyyy-mm-dd hh:mm:ss.0): ").strip()
#try:
#    time_step = float(input("Enter time step (seconds, e.g., 0.1): ") or "0.1")
#except ValueError:
#    time_step = 0.1

pickle_path = '../intersections/SH-55 & Banks-Lowman/Data/DataFrames/df_raw_2025_07_28_1615-2025_07_28_1630.pkl'
output_path = '../output2025-07-28_3.mp4'
shape_csv = '../banks-lowman3.csv'

#video_path = '../vlc-record-2025-07-26-14h35m03s-rtsp___10.37.23.204_axis-media_media.amp-.mp4'
#start_time_str = '2025-07-26 15:35:24.0'
#end_time_str = '2025-07-26 15:37:02.0'

#video_path = '../vlc-record-2025-07-26-14h38m16s-rtsp___10.37.23.204_axis-media_media.amp-.mp4'
#start_time_str = '2025-07-26 15:38:38.0'
#end_time_str = '2025-07-26 15:42:17.0'

#video_path = '../vlc-record-2025-07-28-13h03m55s-rtsp___10.37.23.204_axis-media_media.amp-.mp4'
#start_time_str = '2025-07-28 14:04:20.0'
#end_time_str = '2025-07-28 14:19:17.0'

#video_path = '../vlc-record-2025-07-28-13h20m03s-rtsp___10.37.23.204_axis-media_media.amp-.mp4'
#start_time_str = '2025-07-28 14:20:34.0'
#end_time_str = '2025-07-28 14:26:02.0'

video_path = '../Recording 2025-07-28 152942.mp4'
start_time_str = '2025-07-28 16:23:26.0'
end_time_str = '2025-07-28 16:28:33.0'

time_step=0.2

main(video_path, pickle_path, output_path, shape_csv, start_time_str, end_time_str, time_step)

No shape file found. Starting drawing interface...
Saved 2 shapes to ../banks-lowman3.csv


c:\Users\rhansen\OneDrive - Idaho Transportation Department\Documents\Python\SPMs\Notebooks\spmfunctions\misc_tools.py:215: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_comb.ffill(inplace=True)


KeyboardInterrupt: 